In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import col
from pyspark.sql.functions import lower

spark = SparkSession.builder.appName("StructuredNetworkWordCount").getOrCreate()

In [ ]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
# In command line -> nc -lk 9999

lines =spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()

In [ ]:
# Split the lines into words
words =lines.select(explode(split(lines.value, " ")).alias("word"))

Todas las palabras en minusculas antes de ser contabilizadas, de esta forma las palabras escritas como "Hola", "HOLA", "holA" y "hola", van a ser contabilizadas como una misma palabra "hola". 

In [ ]:
lowers =words.withColumn("lowerWords", lower(col("word")))

In [ ]:
# filtrando los artículos (el, la, los, las, un, una, unos, unas)

filteredWords = lowers.filter(~col("lowerWords").isin(['el', 'la', 'los', 'las', 'un', 'una', 'unos', 'unas']))

In [ ]:
# Generate running word count

wordCounts =filteredWords.groupBy("lowerWords").count()

In [ ]:
# Start running the query that prints the running counts to the console

query = wordCounts.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()